# Load Training Data

In [52]:
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [53]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [54]:
# sub_features = ['43 tGravityAcc-mean()-Z',
#  '52 tGravityAcc-max()-Z',
#  '54 tGravityAcc-min()-Y',
#  '55 tGravityAcc-min()-Z',
#  '56 tGravityAcc-sma()',
#  '59 tGravityAcc-energy()-Z',
#  '125 tBodyGyro-std()-Y',
#  '128 tBodyGyro-mad()-Y',
#  '138 tBodyGyro-energy()-Y',
#  '425 fBodyGyro-mean()-Y',
#  '441 fBodyGyro-energy()-Y',
#  '475 fBodyGyro-bandsEnergy()-1,8',
#  '483 fBodyGyro-bandsEnergy()-1,16',
#  '559 angle(X,gravityMean)',
#  '561 angle(Z,gravityMean)']

# act_features = ['4 tBodyAcc-std()-X',
#  '10 tBodyAcc-max()-X',
#  '202 tBodyAccMag-std()',
#  '215 tGravityAccMag-std()',
#  '269 fBodyAcc-std()-X',
#  '282 fBodyAcc-energy()-X',
#  '303 fBodyAcc-bandsEnergy()-1,8',
#  '311 fBodyAcc-bandsEnergy()-1,16',
#  '315 fBodyAcc-bandsEnergy()-1,24',
#  '504 fBodyAccMag-std()',
#  '505 fBodyAccMag-mad()',
#  '509 fBodyAccMag-energy()']

# input_shape = len(act_features) + len(sub_features)

In [55]:
input_shape

46

In [56]:
train_names = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
train_column_names = train_names.values.tolist()
train_column_names = [k for row in train_column_names for k in row]

train_data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None)
train_data.columns = train_column_names

### Single dataframe column

y_train = pd.read_csv('../data/subject_train.txt', header = None)
y_train.columns = ['Subject']

X_train_1 = train_data[sub_features]
X_train_2 = train_data[act_features]
X_train = pd.concat([X_train_1, X_train_2], axis = 1)

# X_train_1 = train_data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z']
# X_train_2 = train_data.loc[:,'81 tBodyAccJerk-mean()-X':'160 tBodyGyro-correlation()-Y,Z']
# X_train = pd.concat([X_train_1, X_train_2], axis = 1)

X_train = pd.concat([X_train, y_train], axis = 1)
X_train

,42 tGravityAcc-mean()-Y,43 tGravityAcc-mean()-Z,51 tGravityAcc-max()-Y,52 tGravityAcc-max()-Z,54 tGravityAcc-min()-Y,55 tGravityAcc-min()-Z,56 tGravityAcc-sma(),59 tGravityAcc-energy()-Z,125 tBodyGyro-std()-Y,128 tBodyGyro-mad()-Y,...,275 fBodyAcc-max()-X,282 fBodyAcc-energy()-X,"303 fBodyAcc-bandsEnergy()-1,8","311 fBodyAcc-bandsEnergy()-1,16","315 fBodyAcc-bandsEnergy()-1,24",504 fBodyAccMag-std(),505 fBodyAccMag-mad(),506 fBodyAccMag-max(),509 fBodyAccMag-energy(),Subject
0,-0.140840,0.115375,-0.161265,0.124660,-0.123213,0.056483,-0.375426,-0.975510,-0.976623,-0.976353,...,-0.993756,-0.999968,-0.999963,-0.999969,-0.999971,-0.956134,-0.948870,-0.974321,-0.998285,1
1,-0.141551,0.109379,-0.161343,0.122586,-0.114893,0.102764,-0.383430,-0.978500,-0.989046,-0.989038,...,-0.999372,-0.999991,-0.999996,-0.999994,-0.999992,-0.975866,-0.975777,-0.978226,-0.999472,1
2,-0.142010,0.101884,-0.163711,0.094566,-0.114893,0.102764,-0.401602,-0.981672,-0.993552,-0.994122,...,-0.998158,-0.999969,-0.999989,-0.999983,-0.999972,-0.989015,-0.985594,-0.993062,-0.999807,1
3,-0.143976,0.099850,-0.163711,0.093425,-0.121336,0.095753,-0.400278,-0.982420,-0.992407,-0.993142,...,-0.997404,-0.999975,-0.999989,-0.999986,-0.999977,-0.986742,-0.983524,-0.990230,-0.999770,1
4,-0.148750,0.094486,-0.166786,0.091682,-0.121834,0.094059,-0.400477,-0.984363,-0.992378,-0.992542,...,-0.999277,-0.999990,-0.999994,-0.999993,-0.999991,-0.990063,-0.992324,-0.990506,-0.999873,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,-0.222004,-0.039492,-0.214233,-0.016391,-0.234998,-0.071977,-0.405132,-0.995193,0.084878,0.065142,...,-0.318185,-0.674230,-0.684177,-0.666429,-0.668164,-0.232600,-0.007392,-0.401674,-0.584282,30
7348,-0.242054,-0.039863,-0.231477,-0.016391,-0.234998,-0.068919,-0.358934,-0.995151,0.098249,0.091791,...,-0.332146,-0.705580,-0.726986,-0.704444,-0.705435,-0.275373,-0.172448,-0.410577,-0.632536,30
7349,-0.236950,-0.026805,-0.249134,0.024684,-0.216004,-0.068919,-0.377025,-0.995450,0.185902,0.170686,...,-0.160368,-0.692379,-0.655263,-0.674515,-0.684729,-0.220288,-0.216074,-0.362904,-0.641170,30
7350,-0.233230,-0.004984,-0.244267,0.024684,-0.210542,-0.040009,-0.440050,-0.998824,0.190360,0.178939,...,-0.147421,-0.693098,-0.643425,-0.677215,-0.685088,-0.234539,-0.220443,-0.397687,-0.663579,30


In [57]:
X_train = X_train[(X_train['Subject'] == 1) | (X_train['Subject'] == 3) | (X_train['Subject'] == 5)]
X_train = X_train.iloc[:,:-1].values

y_train = y_train[(y_train['Subject'] == 1) | (y_train['Subject'] == 3) | (y_train['Subject'] == 5)]
y_train = y_train.values
y_train = y_train.flatten()

In [58]:
for k in range(len(y_train)):
    if y_train[k] == 1:
        y_train[k] = 0
    elif y_train[k] == 3:
        y_train[k] = 1
    else:
        y_train[k] = 2

# Train Test Data

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.15, shuffle = True)

In [61]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 20),
            classifier_block(20, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [62]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Classifier()
lr = 0.001
n_epochs = 6000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [63]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.541415810585022, Final Batch Loss: 1.1472598314285278
Epoch 2, Loss: 4.529441595077515, Final Batch Loss: 1.1453378200531006
Epoch 3, Loss: 4.474188446998596, Final Batch Loss: 1.0903522968292236
Epoch 4, Loss: 4.488919258117676, Final Batch Loss: 1.1367367506027222
Epoch 5, Loss: 4.45430064201355, Final Batch Loss: 1.1084882020950317
Epoch 6, Loss: 4.4289140701293945, Final Batch Loss: 1.1019666194915771
Epoch 7, Loss: 4.4120999574661255, Final Batch Loss: 1.106124758720398
Epoch 8, Loss: 4.39593780040741, Final Batch Loss: 1.0925155878067017
Epoch 9, Loss: 4.366827845573425, Final Batch Loss: 1.0901483297348022
Epoch 10, Loss: 4.374091386795044, Final Batch Loss: 1.089911937713623
Epoch 11, Loss: 4.367084860801697, Final Batch Loss: 1.0793025493621826
Epoch 12, Loss: 4.340619683265686, Final Batch Loss: 1.0729845762252808
Epoch 13, Loss: 4.339486837387085, Final Batch Loss: 1.0754339694976807
Epoch 14, Loss: 4.313631534576416, Final Batch Loss: 1.0798993110656738
Epo

Epoch 127, Loss: 1.41526859998703, Final Batch Loss: 0.3053182363510132
Epoch 128, Loss: 1.549314171075821, Final Batch Loss: 0.4225519597530365
Epoch 129, Loss: 1.4812469482421875, Final Batch Loss: 0.3625519573688507
Epoch 130, Loss: 1.4026625454425812, Final Batch Loss: 0.2601211965084076
Epoch 131, Loss: 1.4763829708099365, Final Batch Loss: 0.3582393527030945
Epoch 132, Loss: 1.5425027906894684, Final Batch Loss: 0.4359642565250397
Epoch 133, Loss: 1.4171437323093414, Final Batch Loss: 0.33423012495040894
Epoch 134, Loss: 1.485193818807602, Final Batch Loss: 0.3856531083583832
Epoch 135, Loss: 1.4934208691120148, Final Batch Loss: 0.35247594118118286
Epoch 136, Loss: 1.4828131198883057, Final Batch Loss: 0.3113672137260437
Epoch 137, Loss: 1.5495238900184631, Final Batch Loss: 0.5048686861991882
Epoch 138, Loss: 1.4951315820217133, Final Batch Loss: 0.4024078845977783
Epoch 139, Loss: 1.3616831600666046, Final Batch Loss: 0.273358553647995
Epoch 140, Loss: 1.4427745938301086, Fina

Epoch 246, Loss: 1.026938557624817, Final Batch Loss: 0.23699188232421875
Epoch 247, Loss: 1.076637789607048, Final Batch Loss: 0.28577151894569397
Epoch 248, Loss: 1.163891762495041, Final Batch Loss: 0.31891176104545593
Epoch 249, Loss: 1.1598665118217468, Final Batch Loss: 0.28209540247917175
Epoch 250, Loss: 1.0981326252222061, Final Batch Loss: 0.3547370135784149
Epoch 251, Loss: 1.0077074319124222, Final Batch Loss: 0.23815211653709412
Epoch 252, Loss: 1.0844855308532715, Final Batch Loss: 0.2753636837005615
Epoch 253, Loss: 1.0843344926834106, Final Batch Loss: 0.2892037630081177
Epoch 254, Loss: 1.0795336067676544, Final Batch Loss: 0.2259281873703003
Epoch 255, Loss: 1.0032567232847214, Final Batch Loss: 0.2501486539840698
Epoch 256, Loss: 1.0351398438215256, Final Batch Loss: 0.19259421527385712
Epoch 257, Loss: 1.0062847584486008, Final Batch Loss: 0.22555780410766602
Epoch 258, Loss: 1.0250819623470306, Final Batch Loss: 0.30208709836006165
Epoch 259, Loss: 1.02792116999626

Epoch 365, Loss: 0.8521117866039276, Final Batch Loss: 0.2144850790500641
Epoch 366, Loss: 0.8508502840995789, Final Batch Loss: 0.2341218888759613
Epoch 367, Loss: 0.8306499123573303, Final Batch Loss: 0.25149238109588623
Epoch 368, Loss: 0.8330760449171066, Final Batch Loss: 0.16389402747154236
Epoch 369, Loss: 0.8975035101175308, Final Batch Loss: 0.2101123332977295
Epoch 370, Loss: 0.8203452676534653, Final Batch Loss: 0.18085604906082153
Epoch 371, Loss: 0.8204872608184814, Final Batch Loss: 0.1973779946565628
Epoch 372, Loss: 0.8158446550369263, Final Batch Loss: 0.18310502171516418
Epoch 373, Loss: 0.8944858312606812, Final Batch Loss: 0.20851732790470123
Epoch 374, Loss: 0.8073942810297012, Final Batch Loss: 0.15082404017448425
Epoch 375, Loss: 0.8059521019458771, Final Batch Loss: 0.17244410514831543
Epoch 376, Loss: 0.8626739382743835, Final Batch Loss: 0.22453446686267853
Epoch 377, Loss: 0.7810334414243698, Final Batch Loss: 0.12385888397693634
Epoch 378, Loss: 0.8400710374

Epoch 485, Loss: 0.817156508564949, Final Batch Loss: 0.2055555135011673
Epoch 486, Loss: 0.7208766788244247, Final Batch Loss: 0.1410355120897293
Epoch 487, Loss: 0.7718211859464645, Final Batch Loss: 0.20904147624969482
Epoch 488, Loss: 0.7367599904537201, Final Batch Loss: 0.16648942232131958
Epoch 489, Loss: 0.7876281440258026, Final Batch Loss: 0.14673690497875214
Epoch 490, Loss: 0.6811012774705887, Final Batch Loss: 0.11837969720363617
Epoch 491, Loss: 0.773462563753128, Final Batch Loss: 0.17526811361312866
Epoch 492, Loss: 0.8421406149864197, Final Batch Loss: 0.2004801332950592
Epoch 493, Loss: 0.7132490575313568, Final Batch Loss: 0.14176388084888458
Epoch 494, Loss: 0.7038370966911316, Final Batch Loss: 0.11491754651069641
Epoch 495, Loss: 0.6906793266534805, Final Batch Loss: 0.1583898514509201
Epoch 496, Loss: 0.7084178626537323, Final Batch Loss: 0.2027290314435959
Epoch 497, Loss: 0.77812360227108, Final Batch Loss: 0.15115004777908325
Epoch 498, Loss: 0.754370540380477

...

Epoch 5464, Loss: 0.23463325947523117, Final Batch Loss: 0.043818991631269455
Epoch 5465, Loss: 0.16121982224285603, Final Batch Loss: 0.022872421890497208
Epoch 5466, Loss: 0.16176737844944, Final Batch Loss: 0.028585055842995644
Epoch 5467, Loss: 0.25120851024985313, Final Batch Loss: 0.07961469888687134
Epoch 5468, Loss: 0.18176846951246262, Final Batch Loss: 0.019001279026269913
Epoch 5469, Loss: 0.20749759674072266, Final Batch Loss: 0.04386976733803749
Epoch 5470, Loss: 0.24547003209590912, Final Batch Loss: 0.060237277299165726
Epoch 5471, Loss: 0.24028539471328259, Final Batch Loss: 0.06021440401673317
Epoch 5472, Loss: 0.22027422860264778, Final Batch Loss: 0.08828737586736679
Epoch 5473, Loss: 0.20429330505430698, Final Batch Loss: 0.015753908082842827
Epoch 5474, Loss: 0.14149109460413456, Final Batch Loss: 0.010704727843403816
Epoch 5475, Loss: 0.27443885803222656, Final Batch Loss: 0.14170032739639282
Epoch 5476, Loss: 0.20399052277207375, Final Batch Loss: 0.0776001885533

Epoch 5576, Loss: 0.135127454996109, Final Batch Loss: 0.020425470545887947
Epoch 5577, Loss: 0.2011545542627573, Final Batch Loss: 0.03305598720908165
Epoch 5578, Loss: 0.18741981126368046, Final Batch Loss: 0.02447081170976162
Epoch 5579, Loss: 0.22167128883302212, Final Batch Loss: 0.0983002632856369
Epoch 5580, Loss: 0.165477829053998, Final Batch Loss: 0.02525254152715206
Epoch 5581, Loss: 0.0974983493797481, Final Batch Loss: 0.007283331360667944
Epoch 5582, Loss: 0.08495672699064016, Final Batch Loss: 0.017496097832918167
Epoch 5583, Loss: 0.24350832775235176, Final Batch Loss: 0.11799997836351395
Epoch 5584, Loss: 0.15967200137674809, Final Batch Loss: 0.03633945435285568
Epoch 5585, Loss: 0.20911740604788065, Final Batch Loss: 0.01557211671024561
Epoch 5586, Loss: 0.1501067839562893, Final Batch Loss: 0.03676991909742355
Epoch 5587, Loss: 0.19637829065322876, Final Batch Loss: 0.033879391849040985
Epoch 5588, Loss: 0.15399694629013538, Final Batch Loss: 0.014083398506045341
Ep

Epoch 5686, Loss: 0.12069492414593697, Final Batch Loss: 0.014601973816752434
Epoch 5687, Loss: 0.1670805588364601, Final Batch Loss: 0.011666929349303246
Epoch 5688, Loss: 0.35122474655508995, Final Batch Loss: 0.14882834255695343
Epoch 5689, Loss: 0.17283951304852962, Final Batch Loss: 0.02748151309788227
Epoch 5690, Loss: 0.209949541836977, Final Batch Loss: 0.023440483957529068
Epoch 5691, Loss: 0.20871398597955704, Final Batch Loss: 0.030506474897265434
Epoch 5692, Loss: 0.17135434597730637, Final Batch Loss: 0.03878966346383095
Epoch 5693, Loss: 0.17619162052869797, Final Batch Loss: 0.04846421256661415
Epoch 5694, Loss: 0.21005123388022184, Final Batch Loss: 0.01091527845710516
Epoch 5695, Loss: 0.20146027952432632, Final Batch Loss: 0.0842362567782402
Epoch 5696, Loss: 0.1590726375579834, Final Batch Loss: 0.031166819855570793
Epoch 5697, Loss: 0.23597786575555801, Final Batch Loss: 0.05998056381940842
Epoch 5698, Loss: 0.3028542138636112, Final Batch Loss: 0.05110543593764305


Epoch 5800, Loss: 0.17361157294362783, Final Batch Loss: 0.013685797341167927
Epoch 5801, Loss: 0.13745164684951305, Final Batch Loss: 0.025640422478318214
Epoch 5802, Loss: 0.16290271282196045, Final Batch Loss: 0.027814071625471115
Epoch 5803, Loss: 0.20342903211712837, Final Batch Loss: 0.033628661185503006
Epoch 5804, Loss: 0.18842872977256775, Final Batch Loss: 0.03652399033308029
Epoch 5805, Loss: 0.15612060576677322, Final Batch Loss: 0.036980677396059036
Epoch 5806, Loss: 0.20637176930904388, Final Batch Loss: 0.05928882583975792
Epoch 5807, Loss: 0.17564366105943918, Final Batch Loss: 0.015191220678389072
Epoch 5808, Loss: 0.11708669364452362, Final Batch Loss: 0.01986921951174736
Epoch 5809, Loss: 0.1316557638347149, Final Batch Loss: 0.022190233692526817
Epoch 5810, Loss: 0.39964500442147255, Final Batch Loss: 0.13158877193927765
Epoch 5811, Loss: 0.15737557038664818, Final Batch Loss: 0.01510433480143547
Epoch 5812, Loss: 0.16594167053699493, Final Batch Loss: 0.02592949569

Epoch 5910, Loss: 0.18431279063224792, Final Batch Loss: 0.030321799218654633
Epoch 5911, Loss: 0.15181222930550575, Final Batch Loss: 0.03192596882581711
Epoch 5912, Loss: 0.15518302842974663, Final Batch Loss: 0.04111022874712944
Epoch 5913, Loss: 0.1851642206311226, Final Batch Loss: 0.06146009638905525
Epoch 5914, Loss: 0.1932877255603671, Final Batch Loss: 0.009058143012225628
Epoch 5915, Loss: 0.2168816141784191, Final Batch Loss: 0.04608163237571716
Epoch 5916, Loss: 0.21511850133538246, Final Batch Loss: 0.08368702977895737
Epoch 5917, Loss: 0.13270817324519157, Final Batch Loss: 0.030231934040784836
Epoch 5918, Loss: 0.2876902334392071, Final Batch Loss: 0.11376888304948807
Epoch 5919, Loss: 0.17097492888569832, Final Batch Loss: 0.05450289696455002
Epoch 5920, Loss: 0.16073672473430634, Final Batch Loss: 0.022203076630830765
Epoch 5921, Loss: 0.14362961426377296, Final Batch Loss: 0.05243454873561859
Epoch 5922, Loss: 0.1281035579741001, Final Batch Loss: 0.01624235138297081


In [86]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[40  2  0]
 [ 1 54  0]
 [ 0  0 52]]
              precision    recall  f1-score   support

           0      0.976     0.952     0.964        42
           1      0.964     0.982     0.973        55
           2      1.000     1.000     1.000        52

    accuracy                          0.980       149
   macro avg      0.980     0.978     0.979       149
weighted avg      0.980     0.980     0.980       149



In [87]:
torch.save(model.state_dict(), f'../saved_models/UCI 3 User Classifier')

In [60]:
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(metrics.classification_report(y_test, y_pred, digits = 3)) 

C:\Users\hkimr\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0      1.000     0.976     0.988        42
           1      0.982     1.000     0.991        55
           2      1.000     1.000     1.000        52

    accuracy                          0.993       149
   macro avg      0.994     0.992     0.993       149
weighted avg      0.993     0.993     0.993       149

